In [20]:
# import dependencies
# import pandas as pd
# import numpy as np
import random
import json


In [21]:
# select desired state
state = 'missouri'

# access json file with data to be sonified for avaliable states
stateData = json.load(open('../resources/stateData.json'))

# display dictionary
stateData


{'tennessee': {'white': {'population': 5200531,
   'inverted_ccvi': 45.91,
   'chance_of_infection': 15.44,
   'chance_of_death': 1.47,
   'generated_cases': 0,
   'generated_deaths': 0},
  'black': {'population': 1120221,
   'inverted_ccvi': 28.16,
   'chance_of_infection': 15.65,
   'chance_of_death': 1.68,
   'generated_cases': 0,
   'generated_deaths': 0},
  'native': {'population': 18151,
   'inverted_ccvi': 40.66,
   'chance_of_infection': 9.31,
   'chance_of_death': 1.12,
   'generated_cases': 0,
   'generated_deaths': 0},
  'asian': {'population': 117348,
   'inverted_ccvi': 49.7,
   'chance_of_infection': 9.41,
   'chance_of_death': 0.79,
   'generated_cases': 0,
   'generated_deaths': 0},
  'pacific': {'population': 3735,
   'inverted_ccvi': 37.37,
   'chance_of_infection': 24.52,
   'chance_of_death': 1.03,
   'generated_cases': 0,
   'generated_deaths': 0},
  'hispanic': {'population': 363753,
   'inverted_ccvi': 32.45,
   'chance_of_infection': 25.67,
   'chance_of_death':

In [22]:
# list to hold probabilities for which demographic a new case will belong to
infectProbs = []
# list to hold probability that an infection will result in death for each demographic
deathProbs = []

# populate lists above
for data in stateData[state]:
    infectProbs.append(stateData[state][data]['chance_of_infection'])
    deathProbs.append(stateData[state][data]['chance_of_death'])


In [23]:
# weighted probabilities for #people the current case can spread infection to
# (from left to right, values refer to weights for 0, 1, 2, and 3 new infections)
spreadProbs = [0,0,0,12]
# symptomatic cases that have occured
sympCount = 0
# deaths that have occured
deathCount = 0
# case count threshold for when new r0 shoud be calculated
thresh = 0
# nth threshold
n = 1
# infection rate
r0 = 3
# transposition in semitones
trans = 0
# cases remaining (one case to start it off)
more = [0]
# which case
case = 0
# array of midi notes
midiNotes = [48, 51, 55, 58, 62, 63, 65, 67, 68, 68, 74, 77, 78, 80]
# dictionary with all data
weathering = {
    'time': [],
    'demo': [],
    'note': [],
    'death': []
}


In [24]:
# function to determine how many cases correspond to a particular threshold
def nthThresh(n,t):
    return 100 * n + t

# function to adjust spreadProbs weights and calculate r0 for corresponding threshold
def thisThresh():
    if n == 1:
        spreadProbs[3] -= 1
        spreadProbs[2] += 1
    elif n == 2:
        spreadProbs[3] -= 1
        spreadProbs[1] += 1
    elif n == 13:
        spreadProbs[2] -= 1
        spreadProbs[0] += 1
    elif n == 14:
        spreadProbs[1] -= 1
        spreadProbs[0] += 1
    elif n != 0 and n < 13:
        spreadProbs[3] -= 1
        spreadProbs[0] += 1
    r0 = (spreadProbs[1] + 2*spreadProbs[2] + 3*spreadProbs[3]) / sum(spreadProbs)
    
# function to select rescaled incubation period as a delay in milliseconds.
def incubate(n):
    if n == 1:  
        sub = 3
        return random.randint(0, sub*40)*2000/sub
    elif n == 2:
        sub = random.choice([3,4])
        return random.randint(0, sub*40)*2000/sub
    elif n == 3:
        sub = random.choice([6,4])
        return random.randint(0, sub*40)*2000/sub
    elif n == 4:
        sub = random.choice([6,8])
        return random.randint(0, sub*40)*2000/sub
    elif n == 5:
        sub = random.choice([5,6,8])
        return random.randint(0, sub*40)*2000/sub
    else:
        sub = random.choice([5,6,7,8])
        return random.randint(0, sub*40)*2000/sub
    
# function to select a value based on a probability distribution array
def select(probsArray):
    return random.choices(list(range(len(probsArray))),weights=probsArray)[0]

# function to decide between one and zero based on probability that 1 will be chosen
def decide(prob):
    return random.choices([0,1], weights=[100-prob, prob])[0]

# function to generate midi notes
def midi(t):
    note = random.choice(midiNotes)
    return note + t


In [25]:
while len(more) > 0:
    
    # decide how many people the current case infects and add them to more
    for i in range(select(spreadProbs)):
        more.append(case)
        
#     # decide if asymptomatic or not
#     if decide(40) == 0:
        
    # add current case to total # of cases
    # (only sympotmatic cases adjust threshold)
    sympCount += 1

    # if case count is at or over threshold number
    if sympCount >= nthThresh(n, thresh):
        # calculate new threshold
        thresh = nthThresh(n, thresh)
        # adjust spreadProbs weights and calculate r0 for corresponding threshold
        thisThresh()
        # add one to threshold n value
        n += 1

    # select which demographic the current case belongs to
    demo = select(infectProbs)
    weathering['demo'].append(demo)

    # select which note
    weathering['note'].append(midi(0))

    # select incubation period (delay time)
    delay = incubate(1)

    # calculate time from beginning that symptoms occur for current case
    if sympCount == 1:
        time = delay
    else:
        time = delay + weathering['time'][more[0]]
    weathering['time'].append(time)

    # decide/record if case results in death or not
    death = decide(deathProbs[demo])
    weathering['death'].append(death)

    case += 1
        
    # remove current case from how many more cases left
    more.pop(0)
    


In [26]:
result = list(zip(weathering['time'], weathering['demo'], weathering['note'], weathering['death']))

result


[(15333.333333333334, (1, 15333.333333333334), 65, 0),
 (69333.33333333333, (0, 54000.0), 68, 0),
 (15333.333333333334, (5, 0.0), 48, 0),
 (84666.66666666666, (1, 15333.333333333334), 55, 0),
 (61333.333333333336, (1, 46000.0), 77, 0),
 (44666.666666666664, (2, 29333.333333333332), 68, 0),
 (74000.0, (0, 58666.666666666664), 63, 0),
 (45333.333333333336, (5, 30000.0), 77, 0),
 (47333.333333333336, (2, 32000.0), 55, 0),
 (42000.0, (1, 26666.666666666668), 68, 0),
 (67333.33333333333, (0, 52000.0), 65, 0),
 (95333.33333333333, (0, 10666.666666666666), 74, 0),
 (155333.3333333333, (1, 70666.66666666667), 65, 0),
 (126666.66666666666, (5, 42000.0), 77, 0),
 (147333.3333333333, (3, 62666.666666666664), 51, 0),
 (146000.0, (5, 61333.333333333336), 77, 0),
 (88666.66666666667, (0, 27333.333333333332), 74, 0),
 (87333.33333333334, (5, 26000.0), 68, 0),
 (109333.33333333334, (5, 48000.0), 55, 0),
 (102000.0, (2, 40666.666666666664), 55, 0),
 (116000.0, (5, 54666.666666666664), 51, 0),
 (85333.3

In [8]:
result = list(zip(weathering['time'], weathering['demo'], weathering['note'], weathering['death']))

result = sorted(result, reverse=False)

len(result)


11395

In [9]:
result

[(50000.0, (5, 50000.0), 67, 0),
 (86666.66666666666, (1, 36666.666666666664), 80, 0),
 (114666.66666666666, (5, 64666.666666666664), 80, 0),
 (120000.0, (2, 70000.0), 77, 0),
 (121999.99999999999, (2, 7333.333333333333), 80, 0),
 (154000.0, (0, 34000.0), 78, 0),
 (162000.0, (1, 47333.333333333336), 74, 0),
 (173333.33333333334, (1, 11333.333333333334), 78, 0),
 (176000.0, (1, 56000.0), 68, 0),
 (184000.0, (5, 22000.0), 77, 0),
 (193333.33333333334, (1, 39333.333333333336), 55, 0),
 (194666.6666666667, (0, 21333.333333333332), 65, 0),
 (194666.6666666667, (3, 0.0), 63, 0),
 (196000.00000000003, (3, 1333.3333333333333), 67, 0),
 (198666.6666666667, (1, 4000.0), 65, 0),
 (200666.66666666666, (0, 38666.666666666664), 78, 0),
 (201333.33333333334, (3, 6666.666666666667), 68, 0),
 (204000.0, (5, 50000.0), 78, 0),
 (204666.66666666666, (0, 28666.666666666668), 78, 0),
 (204666.66666666666, (5, 28666.666666666668), 80, 0),
 (205333.3333333333, (0, 4666.666666666667), 80, 0),
 (206666.66666666

In [12]:
timing = {}
ms = 0
infections = -1

for time in result:

#     if time[0] != ms:
#         infections += 1
#         timing[infections] = {'demoNotes': [time[1], time[2], time[3]]}
#         timing[infections]['delay'] = time[0] - ms
#         ms = time[0]

#     else:
#         for i in range(1,4):
#             timing[infections]['demoNotes'].append(time[i])

#     if time[0] != ms:
#         timing[time[0]] = {'demoNotes': [time[1], time[2], time[3]]}
#         timing[time[0]]['delay'] = time[0] - ms
#         ms = time[0]

#     else:
#         for i in range(1,4):
#             timing[time[0]]['demoNotes'].append(time[i])


In [13]:
timing

{50000.0: {'demoNotes': [(5, 50000.0), 67, 0], 'delay': 50000.0},
 86666.66666666666: {'demoNotes': [(1, 36666.666666666664), 80, 0],
  'delay': 36666.66666666666},
 114666.66666666666: {'demoNotes': [(5, 64666.666666666664), 80, 0],
  'delay': 28000.0},
 120000.0: {'demoNotes': [(2, 70000.0), 77, 0], 'delay': 5333.333333333343},
 121999.99999999999: {'demoNotes': [(2, 7333.333333333333), 80, 0],
  'delay': 1999.9999999999854},
 154000.0: {'demoNotes': [(0, 34000.0), 78, 0], 'delay': 32000.000000000015},
 162000.0: {'demoNotes': [(1, 47333.333333333336), 74, 0], 'delay': 8000.0},
 173333.33333333334: {'demoNotes': [(1, 11333.333333333334), 78, 0],
  'delay': 11333.333333333343},
 176000.0: {'demoNotes': [(1, 56000.0), 68, 0], 'delay': 2666.666666666657},
 184000.0: {'demoNotes': [(5, 22000.0), 77, 0], 'delay': 8000.0},
 193333.33333333334: {'demoNotes': [(1, 39333.333333333336), 55, 0],
  'delay': 9333.333333333343},
 194666.6666666667: {'demoNotes': [(0, 21333.333333333332),
   65,
  

In [19]:
for time in timing:
    print(f'delay:{timing[time]["demoNotes"][0][1]}, time:{time}')

delay:50000.0, time:50000.0
delay:36666.666666666664, time:86666.66666666666
delay:64666.666666666664, time:114666.66666666666
delay:70000.0, time:120000.0
delay:7333.333333333333, time:121999.99999999999
delay:34000.0, time:154000.0
delay:47333.333333333336, time:162000.0
delay:11333.333333333334, time:173333.33333333334
delay:56000.0, time:176000.0
delay:22000.0, time:184000.0
delay:39333.333333333336, time:193333.33333333334
delay:21333.333333333332, time:194666.6666666667
delay:1333.3333333333333, time:196000.00000000003
delay:4000.0, time:198666.6666666667
delay:38666.666666666664, time:200666.66666666666
delay:6666.666666666667, time:201333.33333333334
delay:50000.0, time:204000.0
delay:28666.666666666668, time:204666.66666666666
delay:4666.666666666667, time:205333.3333333333
delay:2666.6666666666665, time:206666.66666666666
delay:10666.666666666666, time:206666.6666666667
delay:666.6666666666666, time:207333.3333333333
delay:14000.0, time:207333.33333333334
delay:3333.333333333